In [1]:
# import packages and libraries
import pandas as pd
import lightgbm
from xgboost import XGBRegressor
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_validate, cross_val_predict
from sklearn.metrics import mean_squared_error, explained_variance_score
from sklearn.preprocessing import StandardScaler
import numpy as np
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly import tools
import uuid
import os
init_notebook_mode(connected=True)

In [11]:
###help functions###
def PlotCVRes_plotly(X, y, model, cv, scoring, titel):
    dir = os.path.dirname(os.path.abspath("__file__"))
    id = uuid.uuid4().hex
    if titel == '': 
        titel = str(model).replace('\n', '').replace('  ', '')
        titel_new = ''
        pos=0
        com_positions = pd.DataFrame([i for i, ltr in enumerate(titel) if (ltr == ',')])
        split_positions = [com_pos[0] for ind, com_pos in com_positions.iterrows() if ((ind+1)%5==0)] + [len(titel)]
        for ind in split_positions:
            titel_new = titel_new + ' \n ' + titel[pos:ind]
            pos = ind
        titel = titel_new[3:]

    cv_scores = cross_validate(model, np.array(X), np.array(y), cv=cv, scoring=scoring)
    print('MAE_test_mean: ' + str(round(abs(cv_scores['test_MAE']).mean(), 2)) + ', MAE_test_std: ' + str(
        round(abs(cv_scores['test_MAE']).std(), 2)))
    print('RMSE_test_mean: ' + str(round(np.sqrt(abs(cv_scores['test_RMSE'])).mean(), 2)) + ', RMSE_test_std: ' + str(
        round(np.sqrt(abs(cv_scores['test_RMSE'])).std(), 2)))
    print('R2_test_mean: ' + str(round(cv_scores['test_R2'].mean(), 3)) + ', R2_test_std: ' + str(
        round(cv_scores['test_R2'].std(), 3)))
    
    predicted = cross_val_predict(model, X, y, cv=cv)
    #predicted/real plot
    trace1 = go.Scatter(
        x = [y.min(), y.max()],
        y = [y.min(), y.max()],
        name = 'Ideal prediction line',
        line = dict(
            color = ('rgb(0, 0, 0)'),
            width = 3,
            dash = 'dash')
    )
    trace2 = go.Scattergl(
            x = y,
            y = predicted,
            name = 'Model Predictions',
            mode = 'markers',
            marker= dict(size= 3,
                opacity= 0.7,
                color = ('rgb(0, 102, 204)')
               ),
            #text=item_pairs,
            #name=orders,
            #hoverinfo = 'all', #"text + name",
            )

    data = [trace2, trace1]
    
    
    layout = go.Layout(
        xaxis=dict(
            title= 'Real Target Values',
            #showgrid=True,
            #zeroline=True,
            #showline=True,
        ),
        yaxis=dict(
            title= 'Predicted Target Values',
            #showgrid=True,
            #zeroline=True,
            #showline=True,
        ),
        # title="ML Model Cross-validation Results",
        font=dict(family='arial', size=20),
        autosize=False,
        width=600,
        height=600,
        showlegend=False,
        hovermode = "closest",
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig, 'pred_real_scatter')

def Model_validation(X, y, model, scoring, data_stand=False, cv=5, plotCV=True, titel=''):
    if data_stand:
        Xs = StandardScaler().fit_transform(X)
    else:
        Xs = X
    # Plotting Cross-Validated Predictions
    if plotCV:
        PlotCVRes_plotly(Xs, y, model, cv, scoring, titel)
    else:
        cv_scores = cross_validate(model, Xs, y, cv=cv, scoring=scoring)
        print('MAE_test_mean: ' + str(abs(cv_scores['test_MAE']).mean()) + ', MAE_test_std: ' + str(
            abs(cv_scores['test_MAE']).std()))
        print('RMSE_test_mean: ' + str(np.sqrt(abs(cv_scores['test_RMSE'])).mean()) + ', RMSE_test_std: ' + str(
            np.sqrt(abs(cv_scores['test_RMSE'])).std()))
        print('R2_test_mean: ' + str(cv_scores['test_R2'].mean()) + ', R2_test_std: ' + str(
            cv_scores['test_R2'].std()))

In [3]:
data_path_ = r"..\..\01_Data\PUMA_DYNamics\pumadyn-32nm\Dataset.data"
cols = ['theta1', 'theta2',
 'theta3',
 'theta4',
 'theta5',
 'theta6',
 'thetad1',
 'thetad2',
 'thetad3',
 'thetad4',
 'thetad5',
 'thetad6',
 'tau1',
 'tau2',
 'tau3',
 'tau4',
 'tau5',
 'dm1',
 'dm2',
 'dm3',
 'dm4',
 'dm5',
 'da1',
 'da2',
 'da3',
 'da4',
 'da5',
 'db1',
 'db2',
 'db3',
 'db4',
 'db5',
 'thetadd6']
data = pd.read_csv(data_path_, header=None, sep='  ')#.loc[::200,:]
data.columns=cols
data.shape
#data.head()
# data.hist(figsize = (25,10))
# _=plt.show()

C:\ProgramData\Anaconda3\envs\active_ml\lib\site-packages\ipykernel_launcher.py:34: ParserWarning:

Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.



(8192, 33)

In [12]:
# set specific paths
scoring = {'R2': 'r2', 'RMSE': 'neg_mean_squared_error', 'MAE': 'neg_mean_absolute_error'}

X, y = data.iloc[:, :-1], data.iloc[:, -1]

# Shuffel data
np.random.seed(1337)
rng_state = np.random.get_state()
X = np.random.permutation(X)
np.random.set_state(rng_state)
y = np.random.permutation(y)
y=y.ravel()

# lgbmParams = {'boosting_type': 'gbdt',
#                 'n_estimators': 2500,
#                 'objective': 'regression',
#                 'learning_rate': 0.1,
#                 'max_depth': 8,
#                 'num_leaves': 30,
#                 'n_jobs': -1
#               }

# lgbmParams = {'boosting_type': 'gbdt',
#               'random_state': 736283,
#               'num_leaves': 400,
#               'min_data_in_leaf': 5,
#               'max_bin': 500,
#               'learning_rate': 0.08,
#               'n_estimators': 771,
#               'n_jobs': -1,
#               'subsample': 0.8
#               }

# lgbmParams = {'boosting_type': 'gbdt',
#               'random_state': 736283,
#               'num_leaves': 40,
#               'min_data_in_leaf': 5,
#               'max_bin': 500,
#               'learning_rate': 0.08,
#               'n_estimators': 77,
#               'n_jobs': -1,
#               'subsample': 0.8
#               }

# lgbmParams = {'learning_rate': 0.045, 
#               'n_estimators': 581, 
#               'num_leaves': 84, 
#               'objective': 'regression'}

# lgbmParams = {'learning_rate': 0.064, 
#               'n_estimators': 456, 
#               'num_leaves': 97, 
#               'objective': 'regression'}

# model = lightgbm.LGBMRegressor(**lgbmParams)

# xgbParams = {'learning_rate': 0.024,
#              'n_estimators': 580,
#              'max_depth': 6,
#              'missing': -1,
#              'tree_method': 'hist',
#              'min_child_weight': 1,
#              'gamma': 0,
#              'subsample': 0.7,
#              'colsample_bytree': 0.6,
#              'reg_alpha': 1,
#              'scale_pos_weight': 1,
#              'random_state': 736283,
#              'n_jobs': -1}
# model = XGBRegressor(**xgbParams)

xgbParams = {'learning_rate': 0.064, 
              'n_estimators': 1000, 
              'n_jobs': -1}
model = XGBRegressor()

Model_validation(X, y, model, scoring=scoring,
                        cv=3, data_stand=False, 
                        plotCV=True, titel='')

[20:24:02] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:24:04] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:24:06] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
MAE_test_mean: 0.01, MAE_test_std: 0.0
RMSE_test_mean: 0.01, RMSE_test_std: 0.0
R2_test_mean: 0.917, R2_test_std: 0.002
[20:24:08] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:24:10] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[20:24:13] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [6]:
# import pickle
# model.fit(X, y)
# str_mdl = pickle.dumps(model, 0).decode() # dump to model in a string
# #restored_mdl=pickle.loads(pickled_mdl.encode()) #read str object to ml model
# file_object = open('pumadyn_32nm_ML.txt', 'w')
# file_object.write(str_mdl)

# #Read models from string
# file_object = open('pumadyn_32nm_ML.txt', 'r')
# str_mdl = file_object.read()
# pickle.loads(str_mdl.encode())

In [3]:
import pickle
# model.fit(data.iloc[:, :-1], data.iloc[:, -1])
# str_mdl = pickle.dumps(model, 0) # dump to model in a string
# file_object = open('pumadyn_32nm_ML.txt', 'wb')
# file_object.write(str_mdl)

#Read models from string
file_object = open('pumadyn_32nm_ML.txt', 'rb')
str_mdl = file_object.read()
model = pickle.loads(str_mdl)

[18:53:09] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [9]:
model.fit(data.iloc[:, :-1], data.iloc[:, -1])
#validate on the random predictions from this range
actual_lows = {'theta1': -2.3555153, 'theta2': -2.3559486, 'theta3': -2.3556861000000002, 'theta4': -2.3554262, 
                'theta5': -2.355844, 'theta6': -2.3558267999999996, 'thetad1': -2.3556139, 'thetad2': -2.3550772999999996,
                'thetad3': -2.3556497000000003,'thetad4': -2.3553389,'thetad5': -2.3550476000000002,'thetad6': -2.3561332999999998,
                'tau1': -74.990634, 'tau2': -74.937538, 'tau3': -74.978278, 'tau4': -74.999533, 'tau5': -74.984873,
                'dm1': 0.25026488,'dm2': 0.25041709, 'dm3': 0.25022193, 'dm4': 0.25007437, 'dm5': 0.25014939, 'da1': 0.25043561,
                'da2': 0.25057928, 'da3': 0.25008744, 'da4': 0.2501664, 'da5': 0.25061589, 'db1': 0.25005398,
                'db2': 0.25001159, 'db3': 0.25024083, 'db4': 0.25010881, 'db5': 0.2514393}
actual_highs = {'theta1': 2.35, 'theta2': 2.355, 'theta3': 2.355, 'theta4': 2.355, 
                'theta5': 2.35,'theta6': 2.35,'thetad1': 2.35,'thetad2': 2.3540761000000003,'thetad3': 2.3547369,
                'thetad4': 2.3557772,'thetad5': 2.3557997999999998,'thetad6': 2.3558617,'tau1': 74.985591,'tau2': 74.967958,
                'tau3': 74.986797,'tau4': 74.99699100000001,'tau5': 74.995852,'dm1': 2.4999799,'dm2': 2.4994377999999995,
                'dm3': 2.4999333999999998,'dm4': 2.4999981,'dm5': 2.499663,'da1': 2.4991584,'da2': 2.4996680000000002,
                'da3': 2.4999561000000003,'da4': 2.4999662999999996,'da5': 2.4997887999999997,'db1': 2.4999776000000002,
                'db2': 2.4996115000000003,'db3': 2.4998112999999997,'db4': 2.4999968999999997,'db5': 2.4996541000000003}

variables_ = ['theta1', 'theta2', 'theta3', 'theta4', 'theta5', 'theta6', 'thetad1',
            'thetad2', 'thetad3', 'thetad4', 'thetad5', 'thetad6', 'tau1', 'tau2',
            'tau3', 'tau4', 'tau5', 'dm1', 'dm2', 'dm3', 'dm4', 'dm5', 'da1', 'da2',
            'da3', 'da4', 'da5', 'db1', 'db2', 'db3', 'db4', 'db5', 'thetadd6']
samples_ = 10000


def random_generator(samples_, actual_lows, actual_highs, variables_, model):
    df_doe = pd.DataFrame(columns=variables_[:-1])
    for var in variables_[:-1]:
        df_doe[var] = np.random.uniform(actual_lows[var], actual_highs[var],
                                        samples_).round(3)
    df_doe[variables_[-1]] = model.predict(df_doe).round(2)
    return df_doe

def latin_hypercube_generator(samples_, actual_lows, actual_highs, variables, model):
    # actual_lows = {'AT': [2], 'V': [30], 'AP': [993], 'RH': [30]}  # , 'PE':[425]}
    # actual_highs = {'AT': [35], 'V': [80], 'AP': [1033], 'RH': [100]}  # , 'PE':[495]}
    # variables = ['AT', 'V', 'AP', 'RH', 'PE']
    # samples_ = 100000
    # np.random.seed(5234)
    df_doe = pd.DataFrame(lhs(len(variables) - 1, samples=samples_, criterion='maximin'))
    df_doe.columns = variables[:-1]
    for col in df_doe.columns:
        df_doe[col] = [actual_lows[col]] * df_doe.shape[0] + df_doe[col] * (actual_highs[col] - actual_lows[col])
        df_doe[col] = df_doe[col].apply(lambda x: round(x, 3))
    # df_doe = df_doe.reset_index()
    df_doe.columns = variables[:-1]
    df_doe[variables[-1]] = model.predict(df_doe).round(3)
    return df_doe

def normalize(input_array):
    mean = np.mean(input_array, axis=0)
    std = np.std(input_array, axis=0)

    # scikit-learn measure to handle zeros in scale: def _handle_zeros_in_scale(scale, copy=True)
    # https://github.com/scikit-learn/scikit-learn/blob/7389dbac82d362f296dc2746f10e43ffa1615660/sklearn/preprocessing/data.py#L70
    if np.isscalar(std):
        if std == .0:
            std = 1.
    elif isinstance(std, np.ndarray):
        std = std.copy()
        std[std == 0.0] = 1.0

    data_norm = (input_array - mean) / std
    return mean, std, data_norm

df = random_generator(samples_, actual_lows, actual_highs, variables_, model)
print(df['thetad3'].min(), df['thetad3'].max())

C:\ProgramData\Anaconda3\envs\active_ml\lib\site-packages\xgboost\core.py:587: FutureWarning:

Series.base is deprecated and will be removed in a future version

C:\ProgramData\Anaconda3\envs\active_ml\lib\site-packages\xgboost\core.py:588: FutureWarning:

Series.base is deprecated and will be removed in a future version



[18:53:33] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
-2.355 2.355


# Validate Random

In [10]:
from sklearn.linear_model import Ridge
from sklearn.neural_network import MLPRegressor
import lightgbm
from xgboost import XGBRegressor
from sklearn.metrics import r2_score
from pyDOE import lhs

model.fit(data.iloc[:, :-1], data.iloc[:, -1])

size_test_set = 1000
size_train_set = 20
reference_regr = model
np.random.seed(12345)
test_data = latin_hypercube_generator(size_test_set, actual_lows, actual_highs, variables_, reference_regr)
test_data = np.array(test_data)

[18:53:38] WARNING: src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [94]:
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import WhiteKernel, RBF
from sklearn.neighbors import RadiusNeighborsRegressor, KNeighborsRegressor

seedSize = 1000
# rr_params_v1 = {
#     'alpha': 5, 'max_iter': 4, 'normalize': False, 'solver': 'lsqr', 'tol': 0.003}
# predictor = Ridge(**rr_params_v1)
# lgbmParams = {  # 'learning_rate': 0.1,
#     'n_estimators': 100,
#     'objective': 'regression',
#     'n_jobs': 6
#     }
# predictor = lightgbm.LGBMRegressor(**lgbmParams)

# mlp_params_v2 = {
#     'solver': 'adam', 
#     'hidden_layer_sizes': (300, 160, 300, 300, 120, 120, 120), 
#     #'hidden_layer_sizes': (300, 160, 300, 120, 120), 
#     #'early_stopping': True, 
#     'activation': 'tanh', 
#     'tol': 1e-5, 'max_iter': 10000}

# predictor = MLPRegressor(**mlp_params_v2)

#predictor = KNeighborsRegressor(n_neighbors=25, weights='uniform')

# kernel = RBF(length_scale=1.0, length_scale_bounds=(1e-2, 1e3)) \
#          + WhiteKernel(noise_level=1, noise_level_bounds=(1e-10, 1e+1))
# predictor = GaussianProcessRegressor(kernel=kernel)

# lgbmParams = {  # 'learning_rate': 0.1,
#     'n_estimators': 100,
#     'objective': 'regression',
#     'n_jobs': 7,
#     'random_state': 1
#     }
# predictor = lightgbm.LGBMRegressor(**lgbmParams)

np.random.seed(2)
train_data = np.array(random_generator(seedSize, actual_lows, actual_highs, variables_, reference_regr))
mean_data, std_data, data_all_norm = normalize(np.concatenate((train_data, test_data), axis=0))

X_train = train_data[:, :-1]
X_test = test_data[:, :-1]
y_train = train_data[:, -1].reshape((-1, 1))
y_test = test_data[:, -1].reshape((-1, 1))

# NORMALIZED DATA
X_train_norm = data_all_norm[:train_data.shape[0], :-1]
y_train_norm = data_all_norm[:train_data.shape[0], -1].reshape((-1, 1))
X_test_norm = data_all_norm[train_data.shape[0]:, :-1]
y_test_norm = data_all_norm[train_data.shape[0]:, -1].reshape((-1, 1))

#  -------->  MODEL FITTING  <------------------
predictor.fit(X_train_norm, y_train_norm.ravel())
#  -------->  PREDICTION  <------------------
mcs_y_norm = predictor.predict(X_test_norm)

# -------->  DENORMALIZATION OF PREDICTED VALUES  <------------------
mcs_y = mcs_y_norm * std_data[-1].reshape((-1, 1)) + mean_data[-1].reshape((-1, 1))


preds_mcs = np.ravel(mcs_y)
targets_mcs = np.ravel(y_test)

r2_score(targets_mcs, preds_mcs)

0.24498679590741634

lhc

In [34]:
from modAL.models import ActiveLearner
from modAL.models import CommitteeRegressor
from modAL.disagreement import max_std_sampling

numCommitteeMembers = 3
np.random.seed(1 * 100)
train_data = np.array(
    latin_hypercube_generator(seedSize, actual_lows, actual_highs, variables_, reference_regr))

pool_data = np.array(
    latin_hypercube_generator(size_train_set + seedSize, actual_lows, actual_highs, variables_,
                              reference_regr))
# --------> STANDARDIZATION OF DATA SETS (subtraction of the mean, division by standard deviation) <------------------
mean_data, std_data, data_all_norm = normalize(np.concatenate((train_data, pool_data, test_data), axis=0))

X_train = train_data[:, :-1]
X_pool = pool_data[:, :-1]
y_train = train_data[:, -1].reshape((-1, 1))
y_pool = pool_data[:, -1].reshape((-1, 1))
X_test = test_data[:, :-1]
y_test = test_data[:, -1]

# NORMALIZED DATA
X_train_norm = data_all_norm[:train_data.shape[0], :-1]
y_train_norm = data_all_norm[:train_data.shape[0], -1].reshape((-1, 1))
X_pool_norm = data_all_norm[train_data.shape[0]:train_data.shape[0] + pool_data.shape[0], :-1]
y_pool_norm = data_all_norm[train_data.shape[0]:train_data.shape[0] + pool_data.shape[0], -1].reshape((-1, 1))
X_test_norm = data_all_norm[train_data.shape[0] + pool_data.shape[0]:, :-1]
y_test_norm = data_all_norm[train_data.shape[0] + pool_data.shape[0]:, -1].reshape((-1, 1))


# rr_params_v1 = {
#     'alpha': 5, 'max_iter': 4, 'normalize': False, 'solver': 'lsqr', 'tol': 0.003}
# predictor = Ridge(**rr_params_v1)

learner_list=[]
for i in range(numCommitteeMembers):
    learner_list.append(ActiveLearner(estimator=predictor,
                                      # estimator=sklearn.base.clone(predictor),
                                      X_training=X_train_norm,
                                      y_training=y_train_norm.ravel(),
                                      bootstrap_init=True))
committee = CommitteeRegressor(learner_list=learner_list, query_strategy=max_std_sampling)

#  -------->  MODEL FITTING  <------------------
committee.teach(X_train_norm, y_train_norm.ravel())
#  -------->  PREDICTION  <------------------
qbc_y_norm = committee.predict(X_test_norm)

# -------->  DENORMALIZATION OF PREDICTED VALUES  <------------------
qbc_y = qbc_y_norm * std_data[-1].reshape((-1, 1)) + mean_data[-1].reshape((-1, 1))

# -------->  SAVING THE PREDICTED AND TARGET VALUES  <------------------
preds_qbc = np.ravel(qbc_y)
targets_qbc = np.ravel(y_test)


r2_score(targets_qbc, preds_qbc)

0.15615730790279747